<a href="https://colab.research.google.com/github/Shruti-Raj-Vansh-Singh/Chatbot/blob/main/Automated_FAQ_using_word_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Finding answers to FAQ using word embeddings
https://github.com/lavanyats/QuestionAnswering_From_FAQ_Tutorial/blob/master/embeddings_mlinterview_faq.ipynb 
https://www.youtube.com/watch?v=f5GBTOCXoYQ&t=2235s 

#COVID FAQ

In this notebook, we will try to find the most appropriate answer to the question aked by the user form the etof questions provided in the dataset. \
For this we will use word embeddings which are just vectorizedd form of words. We wil compare the the embedding of hte question asked with the once that are mentioned in the dataset and answer using the one that has the maxinmum similiarity using the cosine similarity function. \
The following aret he 4 word embeddings that we will be using are:
1. Word of bags
2. Word2vec
3. GloVe
4. BERT

 \
This is a very basic project and will answer questions using the set of answers given in the dataset. It doesnot generate its own answer.

In [ ]:
#mounting the drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

!unzip '/content/drive/My Drive/dataset/covid_faq.zip'

Archive:  /content/drive/My Drive/dataset/covid_faq.zip
replace covid_faq.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
import pandas as pd
import numpy as np
import re
import os

In [ ]:
dataset = pd.read_csv('covid_faq.csv')

In [ ]:
dataset.head(10)

,questions,answers
0,What is a novel coronavirus?,A novel coronavirus is a new coronavirus that ...
1,Why is the disease being called coronavirus di...,"On February 11, 2020 the World Health Organiza..."
2,How does the virus spread?,The virus that causes COVID-19 is thought to s...
3,Can I get COVID-19 from food (including restau...,Currently there is no evidence that people can...
4,Will warm weather stop the outbreak of COVID-19?,It is not yet known whether weather and temper...
5,What is community spread?,Community spread means people have been infect...
6,Can mosquitoes or ticks spread the virus that ...,"At this time, CDC has no data to suggest that ..."
7,How can I protect myself?,Visit the How to Protect Yourself & Others pag...
8,Does CDC recommend the use of masks to prevent...,Wear masks in public settings when around peop...
9,Is it safe to get care for my other medical co...,It is important to continue taking care of you...


We need to preprocess the data that is provided to us. This can be done by removing contractions, converting everything to lower case, removing punctuation marks, removing stopwords etc. \


In [ ]:
#dictionary for possible contractions
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

In [ ]:
#we will use the nltk library for downloading the stopwords
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop_words = set(stopwords.words('english'))
print(stop_words)

{'once', 'these', 'further', 'himself', 'than', 'will', 'having', 'don', 'but', "you're", 'not', 'doing', 'why', 'll', 'from', 'she', 'd', "that'll", 'when', "hasn't", 'the', 'over', 'very', 'all', 'up', 'hadn', 'during', 'whom', 'haven', 'own', 'was', 'after', 'other', 'being', 'myself', 'again', 'does', 'each', 'who', "you'll", 'what', 'yourselves', 'which', 'no', 'against', "couldn't", 'or', "shouldn't", "hadn't", 'me', 'o', 'mightn', 'by', 'themselves', 've', "won't", 'for', 't', 'shouldn', 'any', 'with', 'hers', "wouldn't", 'they', 'were', 'because', 'off', 'doesn', 'couldn', "aren't", "weren't", 'where', 'aren', 'out', 'most', 'same', 'now', "she's", 'ain', 'too', 'won', 'between', 's', 'before', 'down', 'ours', 'on', 'mustn', 'weren', 'those', 'above', "isn't", 'my', "wasn't", "didn't", 'itself', 'do', 'is', 'only', 'i', 'm', 'such', 'yourself', 'about', "don't", 'our', 'can', 'if', 'wouldn', 'then', 'through', "doesn't", 'yours', 'few', 'them', 'ma', 'in', 'has', 'isn', 'as', '

In this case we need to remove the stopwords, contractions and remove all characeters that are not alphanumeric like punctuations, etc. \
 \
 The function that we define for cleaning our text will have an argument that lets us know if we if we want to keep the stopwords or not. This is important as some embeddings like BERT work well with stopwords, while others moght not give much good results

In [ ]:
def clean_text(text, remove_stopword=True):
  #convert text to lowercase
  newString = text.lower()  
  #remove non alphanumeric characters
  newString = re.sub(r"[^a-zA-Z]"," ", newString)
  #remove contractions
  newString = ' '.join([contraction_mapping[i] if i in contraction_mapping else i for i in newString.split(" ")])

  #removing stopwords
  if remove_stopword==True:
    tokens = [w for w in newString.split() if not w in stop_words]
  else:
    tokens = newString.split()
  long_words = []
  for i in tokens:
    if len(i)>1:
      long_words.append(i)
  return (" ".join(long_words)).strip()

In [ ]:
#define a function to clean all the questions in the dataset
def get_cleaned_sentences(dataset, remove_stopword=False):
  orignal_sentences = dataset['questions']
  cleaned_sentences = []
  for index, row in dataset.iterrows():
    cleaned_sentence = clean_text(row['questions'], remove_stopword)
    cleaned_sentences.append(cleaned_sentence)
  return cleaned_sentences

Let us print the cleaned questions from the dataset \
1. removing stopwords
2. without removing stop words

In [ ]:
#sentences cleaned and containing stop words
cleaned_sentences_with_stopwords = get_cleaned_sentences(dataset, remove_stopword=False)
print(cleaned_sentences_with_stopwords)

#sentences cleaned and not containing stopwords
cleaned_sentences_without_stopwords = get_cleaned_sentences(dataset, remove_stopword=True)
print(cleaned_sentences_without_stopwords)

['what is novel coronavirus', 'why is the disease being called coronavirus disease covid', 'how does the virus spread', 'can get covid from food including restaurant take out produce refrigerated or packaged food or drinking water', 'will warm weather stop the outbreak of covid', 'what is community spread', 'can mosquitoes or ticks spread the virus that causes covid', 'how can protect myself', 'does cdc recommend the use of masks to prevent covid', 'is it safe to get care for my other medical conditions during this time', 'am at risk for covid from mail packages or products', 'is it okay for me to donate blood', 'should contact lens wearers take special precautions to prevent covid', 'is contact lens disinfecting solution effective against covid', 'should use soap and water or hand sanitizer to protect against covid', 'what cleaning products should use to protect against covid', 'what should do if get sick or someone in my house gets sick', 'what should do if have had close contact wit

## 1. Bag of Words (BOW)
Here a vector for the sentence is formed whose length is equal to the number of words in the vocabulary. Each element of the vector represents the number of times a words appears in the sentence.


In [ ]:
sentences = cleaned_sentences_with_stopwords

#split on the basis of white spaces to form the vocabulary
sentence_words = [[word for word in document.split()]for document in sentences]

In [ ]:
from gensim import corpora
#form a dictionary of words: vocabulary
dictionary = corpora.Dictionary(sentence_words)
for key, value in dictionary.items():
  print(key,":",value)

#print the embedding for our dataset
bow_corpus = [dictionary.doc2bow(text) for text in sentence_words]
for sentence, embedding in zip(sentences,bow_corpus):
  print(sentence)
  print(embedding)

0 : coronavirus
1 : is
2 : novel
3 : what
4 : being
5 : called
6 : covid
7 : disease
8 : the
9 : why
10 : does
11 : how
12 : spread
13 : virus
14 : can
15 : drinking
16 : food
17 : from
18 : get
19 : including
20 : or
21 : out
22 : packaged
23 : produce
24 : refrigerated
25 : restaurant
26 : take
27 : water
28 : of
29 : outbreak
30 : stop
31 : warm
32 : weather
33 : will
34 : community
35 : causes
36 : mosquitoes
37 : that
38 : ticks
39 : myself
40 : protect
41 : cdc
42 : masks
43 : prevent
44 : recommend
45 : to
46 : use
47 : care
48 : conditions
49 : during
50 : for
51 : it
52 : medical
53 : my
54 : other
55 : safe
56 : this
57 : time
58 : am
59 : at
60 : mail
61 : packages
62 : products
63 : risk
64 : blood
65 : donate
66 : me
67 : okay
68 : contact
69 : lens
70 : precautions
71 : should
72 : special
73 : wearers
74 : against
75 : disinfecting
76 : effective
77 : solution
78 : and
79 : hand
80 : sanitizer
81 : soap
82 : cleaning
83 : do
84 : gets
85 : house
86 : if
87 : in
88 : sick

In [ ]:
#for intuition
question = 'can i donote blood'
question = clean_text(question,False)
question_embeddings = dictionary.doc2bow(question.split())

print("Question:", question)
print("Embeddig:", question_embeddings)

Question: can donote blood
Embeddig: [(14, 1), (64, 1)]


Now we have the vector for each sentence in the dataset, we can compute the cosine similarity between them to find that one that is most similar to the asked question

In [ ]:
#cosine similarity
from sklearn.metrics.pairwise import cosine_similarity

def FAQ(question, question_embedding, sentence_embedding, dataset, sentences):
  max_similarity=-1
  index_similarity=-1
  for index, faq_embedding in enumerate(sentence_embedding):
    similarity = cosine_similarity(faq_embedding, question_embedding)[0][0]
    #print(index, similarity, sentence[index])
    if similarity>max_similarity:
      max_similarity = similarity
      index_similarity =index


  print("Question:", question)
  print("Predicted question:",  dataset.iloc[index_similarity,0])
  print("Ans: ",dataset.iloc[index_similarity,1])

In [ ]:
#function calling
# question = 'How to prevent corona?'
# question = clean_text(question,False)
# question_embeddings = dictionary.doc2bow(question.split())

FAQ(question, question_embeddings, bow_corpus, dataset, sentences)

## 2. Word2Vec Embedding
It is based on training a shallow feed forward neural network. The relationship between words is derived through cosine similarity.  \
The embeddings are trained to take a word as input and reconstruct its context. Hence we can take into account the sematic similarity of words based on context information. 

link https://code.google.com/archive/p/word2vec/ 

In [ ]:
from gensim.models import Word2Vec
import gensim.downloader as api

v2w_model = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


The embeddings provided ny the word2vec are for individual words. To use these embeddings for phrases/sentences we sum up individual word embeddings in the phrase/sentence to get the sentence vector.

In [ ]:
def word_vector(word, model):
  #there are 2 situations here: the word exists in the embedding or it does not
  samp = model['computer']
  vec = [0]*len(samp)
  try:
    vec = model[word]
  except:
    vec = [0]*len(samp)

  #return the vector for the word embedding
  return vec

In [ ]:
def phrase_embedding(phrase, embedding_model):
  sample = word_vector('computer',v2w_model)
  vec = np.array([0]*len(sample))
  length = 0
  for word in phrase.split():
    length+=1
    vec = vec+np.array(word_vector(word, v2w_model))

  return vec.reshape(1,-1)

In [ ]:
sentence_embeddings=[]
cleaned_sentences = get_cleaned_sentences(dataset, remove_stopword=True)
for sentence in cleaned_sentences:
  sentence_embeddings.append(phrase_embedding(sentence,v2w_model))
question_embedding = phrase_embedding(question, v2w_model)

FAQ(question, question_embedding, sentence_embeddings, dataset, cleaned_sentences)

Question: can donote blood
Predicted question: Is it okay for me to donate blood?
Ans:  In healthcare settings across the United States, donated blood is a lifesaving, essential part of caring for patients. The need for donated blood is constant, and blood centers are open and in urgent need of donations. CDC encourages people who are well to continue to donate blood if they are able, even if they are practicing social distancing because of COVID-19. CDC is supporting blood centers by providing recommendations that will keep donors and staff safe. Examples of these recommendations include spacing donor chairs 6 feet apart, thoroughly adhering to environmental cleaning practices, and encouraging donors to make donation appointments ahead of time.


## 3.GloVe Embedding:


In [ ]:
glove_model = api.load('glove-twitter-25')

[==================================================] 100.0% 104.8/104.8MB downloaded


In [ ]:
sentence_embedding_glove = []
for sentence in cleaned_sentences:
  sentence_embedding_glove.append(phrase_embedding(sentence, glove_model))
question_embedding_glove = phrase_embedding(question, glove_model)

FAQ(question, question_embedding_glove, sentence_embedding_glove, dataset, cleaned_sentences)

Question: can donote blood
Predicted question: Is it okay for me to donate blood?
Ans:  In healthcare settings across the United States, donated blood is a lifesaving, essential part of caring for patients. The need for donated blood is constant, and blood centers are open and in urgent need of donations. CDC encourages people who are well to continue to donate blood if they are able, even if they are practicing social distancing because of COVID-19. CDC is supporting blood centers by providing recommendations that will keep donors and staff safe. Examples of these recommendations include spacing donor chairs 6 feet apart, thoroughly adhering to environmental cleaning practices, and encouraging donors to make donation appointments ahead of time.


## 4. BERT embedding

In [ ]:
from bert_serving.client import BertClient
bc = BertClient()

In [ ]:
sentence_embedding_glove = []
for sentence in cleaned_sentences:
  sentence_embedding_glove.append(phrase_embedding(sentence, bc.encode[sentence]))
question_embedding_glove = phrase_embedding(question, bc.encode[sentence])

FAQ(question, question_embedding_glove, sentence_embedding_glove, dataset, cleaned_sentences)